# Step 0 Creating the data base
Inserting default information


#### This may not still be relevant

In [1]:
import sqlite3

from settings import *

In [2]:
class LoadDatabase():
    
    def __init__(self, dbname, df):
        self.conn = sqlite3.connect(dbname)
        self.cursor = self.conn.cursor()
        self.load_management_chart_of_accounts()
        self.cursor.execute('CREATE TABLE IF NOT EXISTS trial_balance (period text, code integer, balance real)')
        self.df=df
        
    def load_management_chart_of_accounts(self):
        """Assets - Liabilities - OwnersEquity = 0
        Asset - Liability - Equity - Income + Expense = 0
        """
        self.cursor.execute('CREATE TABLE IF NOT EXISTS chart_of_accounts (chart text, code integer, name text, category text)')
        coa_name = 'SLF-MA'
        if self.empty_coa(coa_name):
            coa= {'4000': 'Sales',
            '4009': 'Discounts Allowed',
            '4905': 'Carriage',
            '5000': 'Materials Cost',
            '5001': 'Material Cost Adjustment',
            '6100': 'Commissions',
            '6200': 'Sales Manager',
            '6201': 'Advertising',
            '7000': 'Factory Wages',
            '7006': 'Redundancy Costs',
            '7020': 'Delapidations Provision',
            '7100': 'Factory Gas & Elec',
            '7102': 'Packaging',
            '7103': 'Machine Repairs',
            '7105': 'Works Sundries',
            '7200': 'Factory Depreciation',
            '7202': 'Factory Insurance',
            '7204': 'Factory Rent & Rates',
            '7206': 'Factory Salaries',
            '7906': 'Exchange Rate Variance',
            '8100': 'Bad Debts',
            '8200': 'Audit Fees',
            '8201': 'Legal/Professional',
            '8204': 'Bank Charges/Interest',
            '8300': 'Admin Depreciation',
            '8310': 'Gain/Loss on Disposal of Assets',
            '8400': 'Admin Gas & Electricity',
            '8402': 'Admin Insurance',
            '8405': 'Motor Running',
            '8408': 'Postage',
            '8410': 'Admin Rent & Rates',
            '8414': 'Repairs/Maint/Bldgs',
            '8420': 'Admin Salaries',
            '8424': 'Stationary',
            '8426': 'Telephone',
            '8430': 'Computer/EDI/Internet',
            '8433': 'New Product Development',
            '8435': 'Admin Travel',
            '8440': 'Admin Sundries'}
            for k,v in coa.items():
                self.cursor.execute("INSERT INTO chart_of_accounts (chart, code, name) VALUES ('{}', {}, '{}')".format(coa_name, k, v))
            self.cursor.execute("UPDATE chart_of_accounts SET category='Income' WHERE code in (4000)")
            self.cursor.execute("UPDATE chart_of_accounts SET category='Expense' WHERE code > 4000")

        
    def __enter__(self):
        return self

    
    def __exit__(self, exc_type, exc_value, traceback):
        self.commit()
        self.close()
 

    def commit(self):
        self.conn.commit()

        
    def close(self):
        self.conn.close()
 

    def empty_coa(self, coa):
        """Check if this chart of management accounts hasn't been entered."""
        count=self.cursor.execute("SELECT COUNT(*) FROM chart_of_accounts WHERE chart='{}'".format(coa)).fetchone()[0]
        return count==0

    
    def empty(self, period):
        """Check if not data for period is in database"""
        count=self.cursor.execute("SELECT COUNT(*) FROM trial_balance WHERE period='{}'".format(period)).fetchone()[0]
        return count==0

    
    def load_mtd_to_database(self, period):
        if self.empty(period):
            s = self.df['PLTD-{}'.format(period)]
            l = s.index.tolist()
            for i,v in enumerate(s):
                code = str(l[i])
                # print(l[i], type(l[i]), str(l[i]), v)
                # Insert a row of data
                if v == '-' or math.isnan(v):
                    v = p(0)
                else:
                    v = p(v)
                process_normally = not( (code=='5001' and (v == p(0))))
                if process_normally:                
                    if code == '4000':
                        v = -v
                    self.cursor.execute("INSERT INTO trial_balance VALUES ('{}', {}, {})".format(period, code, v))
        else:
            raise LoadDatabaseError('{} already is in MTD database'.format(period))

## Extending the database definition
Want to have a summary table for each set of trial balances.  This should have some text such as what do the suffixes mean.
Perhaps Inception data shoul
Each Entry should have more information such as Period start and end, Balance sheet start and end

In [4]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS company 
(name text, description text, inception_date text, extra_data text, company_number text)""")

def empty(company):
    """Check if this chart of management accounts hasn't been entered."""
    count=cursor.execute("SELECT COUNT(*) FROM company WHERE name='{}'".format(company)).fetchone()[0]
    return count==0

default = [('Slumberfleece', '02235270', '1988-03-24')]

for company, number, inception_date in default:
    if empty(company):
        print('Insert company data for {}'.format(company))
        sql = """INSERT INTO company 
(name, description, inception_date, extra_data, company_number)
VALUES('{0}', '# {0} Trial Balance Information\nThis is a list of the trial balance naming convention used.',
       '{2}', '---\n- {0}  # Place to store additional info.\n...\n', '{1}')""".format(
                    company, number, inception_date)
        print(sql)
        cursor.execute(sql)
conn.commit
conn.close

Insert company data for Slumberfleece
INSERT INTO company 
(name, description, inception_date, extra_data, company_number)
VALUES('Slumberfleece', '# Slumberfleece Trial Balance Information
This is a list of the trial balance naming convention used.',
       '1988-03-24', '---
- Slumberfleece  # Place to store additional info.
...
', '02235270')


OperationalError: database is locked